In [59]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import pandas as pd
from collections import Counter
import torchtext.vocab as Vocab
import re
import random
import numpy as np
from tqdm import trange

In [2]:
def set_random_seeds(seed=0):
    """
    Set random seeds for all possible random variable generators.
    :param seed: The given random seed (default 0)
    :return: None
    """
    random.seed(seed)  					# 为Python设置随机种子
    np.random.seed(seed)  				# 为Numpy设置随机种子
    torch.manual_seed(seed)  			# 为CPU设置随机种子
    torch.cuda.manual_seed(seed)  		# 为当前GPU设置随机种子
    torch.cuda.manual_seed_all(seed)  	# 为所有GPU设置随机种子

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seq_len = 10
batch_size = 16
embed_dim = 100
learning_rate = 2e-1
hidden_size = 200
num_layers = 2
clipping_theta = 1e-2
num_epochs = 50
set_random_seeds(189)

In [4]:
def get_data(texts_path, labels_path):
    texts = pd.read_csv(texts_path, header=0, names=['ID', 'Sentence 0', 'Sentence 1'])
    labels = pd.read_csv(labels_path, header=None, names=['ID', 'Answer'])['Answer']
    return texts, labels

In [5]:
train_texts, train_labels = get_data(
    '../data/Training Data/subtaskA_data_all.csv',
    '../data/Training Data/subtaskA_answers_all.csv'
)

val_texts, val_labels = get_data(
    '../data/Dev Data/subtaskA_dev_data.csv',
    '../data/Dev Data/subtaskA_gold_answers.csv'
)

test_texts, test_labels = get_data(
    '../data/Test Data/subtaskA_test_data.csv',
    '../data/Test Data/subtaskA_gold_answers.csv'
)

# Use the validation dataset as the training dataset
train_texts = pd.concat([train_texts, val_texts], axis=0, ignore_index=True)
train_labels = pd.concat([train_labels, val_labels], axis=0, ignore_index=True)
train_data = pd.concat([train_texts, train_labels], axis=1)
texts = pd.concat([train_data["Sentence 0"], train_data["Sentence 1"]], axis=0, ignore_index=True)
texts = [re.sub(r'[^0-9a-zA-Z ]', '', text) for text in texts]
train_sents = [train_data.iloc[i, 1] if train_data.iloc[i, 3] == 1 else train_data.iloc[i, 2] for i in range(len(train_data))]

In [6]:
def tokenize(text):
    return text.lower().split()

class Vocabulary:
    def __init__(self, texts, min_freq=2, vocabulary=None):
        self.tok2id = None
        self.id2tok = None
        self.size = None
        self.get_vocab(texts, min_freq, vocabulary)

    def get_vocab(self, texts, min_freq, vocabulary=None):
        if vocabulary is None:
            tokens_list = [tokenize(text) for text in texts]
            counter = Counter([token for tokens in tokens_list for token in tokens])
            id_set = {token for token, _ in filter(lambda x: x[1] >= min_freq, counter.items())}
            self.tok2id = {token: index for index, token in enumerate(["[PAD]", "[UNK]"] + sorted(id_set))}
        else:
            self.tok2id = vocabulary
        self.id2tok = list(self.tok2id)
        self.size = len(self.id2tok)

    def tokens2idx(self, tokens, seq_len):
        tokens_pad = tokens[:seq_len] if len(tokens) > seq_len else tokens + ["[PAD]"] * (seq_len - len(tokens))
        return torch.tensor([self.tok2id.get(token, self.tok2id["[UNK]"]) for token in tokens_pad])

In [7]:
vocab = Vocabulary(texts, min_freq=3)  # 建立词典
print(f"词典中共{vocab.size}个词。")  

词典中共4415个词。


In [8]:
train_X = []
for sent in train_sents:
    tokens = [token for token in tokenize(sent) if token != "."]
    train_X.append(vocab.tokens2idx(tokens, seq_len))
train_Y = torch.stack([torch.cat([item[1:], torch.tensor([0])]) for item in train_X])
train_X = torch.stack(train_X)
dataset = Data.TensorDataset(train_X, train_Y)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [9]:
def load_pretrained_embedding(words, pretrained_vocab):
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0])
    oov_count = 0  # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    print(f"There are {oov_count} oov words.")
    return embed

In [10]:
glove_vocab = Vocab.GloVe(name='6B', dim=embed_dim, cache="../pretrained_models/glove.6B")
pre_embedding = load_pretrained_embedding(vocab.id2tok, glove_vocab)

There are 18 oov words.


In [11]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, dropout_rate=0.15):
        super(LSTM, self).__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.rnn = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_size,
            num_layers=num_layers
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features=hidden_size, out_features=vocab_size)
        )

    def read_pre_embedding(self, pre_embedding):
        self.embed.weight.data.copy_(pre_embedding)
#         self.embed.weight.requires_grad = False

    def forward(self, X):                                       # (batch_size, seq_len)
        X_embed = self.embed(X.permute(1, 0))  					# (seq_len, batch_size, embed_dim)
        X_rnn, _ = self.rnn(X_embed)  							# (seq_len, batch_size, hidden_size)
        outs = self.fc(X_rnn.view(-1, hidden_size))             # (batch_size * seq_len, vocab_size)
        return outs                                             # (batch_size * seq_len, vocab_size)
    

In [12]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)

        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [13]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        if param.requires_grad:
            norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            if param.requires_grad:
                param.grad.data *= (theta / norm)


In [15]:
model = LSTM(vocab.size, embed_dim, hidden_size, num_layers).to(device)
model.read_pre_embedding(pre_embedding)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
for epoch in range(num_epochs):
    for X, Y in data_iter:
        X = X.to(device)
        Y = Y.view(-1).to(device)
        output = model(X) # output: 形状为(num_steps * batch_size, vocab_size)
        loss = criterion(output, Y)
        optimizer.zero_grad()
        loss.backward()
        grad_clipping(model.parameters(), clipping_theta, device)
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch+1}\tloss: {loss.item()}")

Epoch: 10	loss: 3.9485023021698
Epoch: 20	loss: 4.559276580810547
Epoch: 30	loss: 5.404726028442383
Epoch: 40	loss: 4.505176544189453
Epoch: 50	loss: 4.454569339752197


In [45]:
def score(model, sentence):
    tokens = [token for token in tokenize(sentence) if token != "."]
    X = vocab.tokens2idx(tokens, seq_len)
    Y = torch.cat([X[1:], torch.tensor([0])])
    X = X.unsqueeze(0).to(device)
    Y = Y.to(device)
    with torch.no_grad():
        outputs = model(inputs)
    return criterion(outputs, Y).item()

In [61]:
num_correct = 0
num_total = len(test_texts)
for i in trange(num_total):
    label = test_labels[i]
    sentences = test_texts.iloc[0, 1:].tolist()
    scores = np.array([score(model, sentence) for sentence in sentences])
    answer = np.argmax(scores)
    if answer == label:
        num_correct += 1

print(f"correct/total: {num_correct}/{num_total}")
print(f"Accuracy: {num_correct / num_total}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 397.04it/s]

correct/total: 508/1000
Accuracy: 0.508
